In [20]:
import pandas as pd

# Load data from CSV files
data_2022_2023 = pd.read_csv('../data/football_data_2022_2023_raw.csv')
data_2023_2024 = pd.read_csv('../data/football_data_2023_2024_raw.csv')
data_2024_2025 = pd.read_csv('../data/football_data_2024_2025_raw.csv')

# Combine all seasons into one DataFrame
data = pd.concat([ data_2022_2023,data_2023_2024, data_2024_2025])

# Inspect the first few rows
data.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,1XBCH,1XBCD,1XBCA,BFECH,BFECD,BFECA,BFEC>2.5,BFEC<2.5,BFECAHH,BFECAHA
0,E0,05/08/2022,20:00,Crystal Palace,Arsenal,0,2,A,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,06/08/2022,12:30,Fulham,Liverpool,2,2,D,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,06/08/2022,15:00,Bournemouth,Aston Villa,2,0,H,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,06/08/2022,15:00,Leeds,Wolves,2,1,H,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,06/08/2022,15:00,Newcastle,Nott'm Forest,2,0,H,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Handle missing data

# Check for missing values
data.isnull().sum()

# Drop rows with missing target columns (e.g., missing goals)
data = data.dropna(subset=['FTHG', 'FTAG'])  # FTHG: Full-time Home Goals, FTAG: Full-time Away Goals

# Fill missing numerical values with the column mean (for example, possession or shots on target)
data['HS'] = data['HS'].fillna(data['HS'].mean())  # HS: Home Shots
data['AS'] = data['AS'].fillna(data['AS'].mean())  # AS: Away Shots

In [22]:
# Convert the date column to datetime format
data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')

# Convert categorical columns (e.g., home and away teams) to category type
data['HomeTeam'] = data['HomeTeam'].astype('category')
data['AwayTeam'] = data['AwayTeam'].astype('category')


In [23]:
# Add goal difference feature
data['GoalDifference'] = data['FTHG'] - data['FTAG']  # FTHG: Home Goals, FTAG: Away Goals


In [24]:
# A more advanced feature engineering method would calculate rolling averages or sums for each team
# (e.g., rolling average of goals scored in the last 5 matches)

# Before using categorical features (e.g., team names) in machine learning, they need to be encoded. A common method is One-Hot Encoding
# One-Hot Encode categorical features (like teams)
# pd.get_dummies() encodes categorical variables into binary columns. For example, if there are 20 teams, it will create 19 binary columns (one per team, minus one to avoid redundancy).
# drop_first=True ensures that we drop one of the categories to prevent multicollinearity.
data_encoded = pd.get_dummies(data, columns=['HomeTeam', 'AwayTeam'], drop_first=False)

In [25]:
# Save the cleaned data
data_encoded.to_csv('../data/football_data_cleaned.csv', index=False)

In [26]:
# Result field was missed from football_data_cleaned. 

# Load the cleaned dataset
data = pd.read_csv('../data/football_data_cleaned.csv')

# Create the 'Result' column
data['Result'] = data.apply(lambda row: 'Home Win' if row['FTHG'] > row['FTAG'] else ('Away Win' if row['FTAG'] > row['FTHG'] else 'Draw'), axis=1)

# Check the first few rows to ensure the 'Result' column is added
data[['FTHG', 'FTAG', 'Result']].head()

,FTHG,FTAG,Result
0,0,2,Away Win
1,2,2,Draw
2,2,0,Home Win
3,2,1,Home Win
4,2,0,Home Win


In [27]:
# Map 'Result' to numerical values
result_mapping = {'Home Win': 2, 'Draw': 1, 'Away Win': 0}
data['Result'] = data['Result'].map(result_mapping)

# Check the encoding
data[['FTHG', 'FTAG', 'Result']].head()

,FTHG,FTAG,Result
0,0,2,0
1,2,2,1
2,2,0,2
3,2,1,2
4,2,0,2


In [28]:
# Save the updated dataset with the 'Result' column
data.to_csv('../data/football_data_cleaned.csv', index=False)